## Perceptrons

<div style="display: flex; flex-direction: row;">
        <div style="flex: 3; padding: 20px; margin: 10px">

- A perceptron takes several binary inputs, $x_1$, $x_2$, ... and produces a single binary output.

- <a href="https://en.wikipedia.org/wiki/Frank_Rosenblatt">Frank Rosenblatt</a> introduced *weights*, real numbers expressing the importance of the inputs.

- The neuron's output, 0 or 1, is determined by whether the weighted sum 
$\sum_jw_jx_j$ is less than or greater than some threshold value ($b$). 


</div>
        <div style="flex: 1.5; background-color: #f6f6f6; border-radius: 20px; padding: 20px; margin: 10px">
        <img src="http://neuralnetworksanddeeplearning.com/images/tikz0.png"><br>
        <div style="text-align: right"><a href="http://neuralnetworksanddeeplearning.com/chap1.html">source</a></div>
        </div>
</div>

## Briefly about logical gates

In [1]:
# AND gate returns True only when its input are True.
def and_gate(a, b):
    if a == True and b == True:
          return True
    else: return False

# NAND gate returns False only when its inputs are True.
# In other words, a NAND gate is the negation of an AND gate.
def nand_gate(a, b):
    if a == True and b == True:
          return False
    else: return True

# XOR gate returns True only when its inputs are different.
def xor_gate(a, b):
    if a != b:
          return True
    else: return False

In [2]:
print(f'{nand_gate(0, 1)=}, {xor_gate(0,1)=}')
print(f'{nand_gate(0, 0)=}, {xor_gate(0,0)=}')

nand_gate(0, 1)=True, xor_gate(0,1)=True
nand_gate(0, 0)=True, xor_gate(0,0)=False


Suppose we have a perceptron with two inputs, each with weight −2 and an overall bias of 3:<br>
<div style="display: flex; flex-direction: row;">
<div style="flex: 1.5; background-color: #f6f6f6; padding: 20px; margin: 10px; border-radius:20px;">
<img src="http://neuralnetworksanddeeplearning.com/images/tikz2.png"><br>
<div style="text-align: right"><a href="http://neuralnetworksanddeeplearning.com/chap1.html">source</a></div>
</div>
<div style="flex: 3; padding: 20px; margin: 10px">

We see that input 0 0 produces ouput 1, since:

<code>(-2) * 0 + (-2) * 0 + 3 = 3</code> is positive. 

While input 1 1 produces output -1, since:

<code>(-2) * 1 + (-2) * 1 + 3 = -1</code> is negative.

And so our perceptron implements a NAND gate!

</div></div>

We can use networks of perceptrons to compute any logical function.


---

## Creating a simplified (broken) ANN

Let's say we want to create a neural network that classifies if a customer will buy a product based on their age and income.

In [3]:
from sklearn.datasets import make_classification
import pandas as pd

# Generate pre-engineered synthetic data
X, y = make_classification(n_samples=1000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, random_state=42)

data = pd.DataFrame(X, columns=['age', 'income'])
data['target'] = y

In [4]:
data.head()

,age,income,target
0,-0.999102,-0.663860,1
1,1.246686,1.153597,1
2,0.962777,0.859397,1
3,-2.957441,2.033645,1
4,1.141165,1.059449,1


We use the sigmoid activation function and a neural network with one hidden layer and two neurons.

In [5]:
from keras.models import Sequential     # Linear stack of layers
from keras.layers import Dense          # Dense layer for fully connected layers

# Feedforward neural network
model = Sequential()

# Add a Dense layer with 2 output units and sigmoid activation function to represent the hidden layer
model.add(Dense(2, input_dim=2, activation='sigmoid'))

# Add another Dense layer with 1 output unit and sigmoid activation function to represent the output layer
model.add(Dense(1, activation='sigmoid'))

# Cross-entropy loss function: measures the performance of the model
# Adam optimizer: updates the weights and accuracy metric to evaluate the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-03-28 15:27:08.036499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


We train the model using the fit() method in Keras, and use a validation split of 20% to monitor the performance of the model during training.

In [6]:
# Train the model
# history = model.fit(X, y, epochs=10, validation_split=0.2, verbose=1)

In [7]:
# Increase epochs
history = model.fit(X, y, epochs=100, validation_split=0.2, verbose=0)

Evaluate the performance of the neural network:

In [8]:
# Evaluate the performance of the model on the test set
score = model.evaluate(X, y, verbose=1)
print("Accuracy: %.2f%%" % (score[1]*100))

32/32 [==============================] - 0s 1ms/step - loss: 0.3477 - accuracy: 0.8680
Accuracy: 86.80%


Fine-tune the hyperparameters:

In [9]:
from keras.optimizers import Adam

# Define the neural network architecture with new hyperparameters
model = Sequential()
model.add(Dense(2, input_dim=2, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model with a lower learning rate
optimizer = Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train the model with more epochs
history = model.fit(X, y, epochs=500, validation_split=0.2, verbose=0)

Make predictions on new data:

In [10]:
X_test, y_test = make_classification(n_samples=10, n_features=2, n_informative=2, n_redundant=0, n_classes=2, random_state=42)

# Create a pandas dataframe from the data
test_data = pd.DataFrame(X_test, columns=['age', 'income'])

In [11]:
test_data, y_test

(        age    income
 0  1.068339 -0.970073
 1 -1.140215 -0.838792
 2 -2.895397  1.976862
 3 -0.720634 -0.960593
 4 -1.962874 -0.992251
 5 -0.938205 -0.543048
 6  1.727259 -1.185827
 7  1.777367  1.511576
 8  1.899693  0.834445
 9 -0.587231 -1.971718,
 array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0]))

In [12]:
model.predict(test_data)

1/1 [==============================] - 0s 85ms/step


array([[0.07173289],
       [0.20864128],
       [0.955787  ],
       [0.1380484 ],
       [0.22591664],
       [0.32604253],
       [0.04844391],
       [0.952324  ],
       [0.9014092 ],
       [0.03739632]], dtype=float32)